# Chat with Notion databases.

prerequisite:

1. OpenAI API Key.
2. Create a Notion integration.
3. You need to obtain a Notion database ID and grant authorization to the above integration for that database.

An example of the configuration file `.notion_to_chatbot.ini` is as follows:

```
[Notion]
integration_token = your_integration_token
database_id = your_notion_database_id
[OpenAI]
openai_api_key = your_openai_api_key
```

In [ ]:
!pip3 install langchain openai chromadb tiktoken

In [3]:
import configparser
from langchain.document_loaders import NotionDBLoader

config = configparser.ConfigParser()
config.read('/Users/futeng/Documents/OneDrive/workspaces/key/.notion_to_chatbot.ini')  

integration_token = config['Notion']['integration_token']
database_id = config['Notion']['database_id']

loader = NotionDBLoader(
    integration_token=integration_token,
    database_id=database_id,
    request_timeout_sec=30,  # optional, defaults to 10
)

docs = loader.load()

In [4]:
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import os
from langchain.docstore.document import Document
openai_api_key = config['OpenAI']['openai_api_key']
os.environ["OPENAI_API_KEY"] = openai_api_key

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
texts = [Document(page_content=x.page_content) for x in texts]

Created a chunk of size 2124, which is longer than the specified 1000
Created a chunk of size 2884, which is longer than the specified 1000


In [5]:
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever())

100%|██████████| 1/1 [00:01<00:00,  1.45s/it]


In [10]:
qa.run("我的DNS 域名服务器是多少？")

' The DNS domain name server is 10.0.0.1.'